# Лекция 10: Логические методы и Деревья Решений

**Цель лекции:**
1. Познакомиться с интуицией **логических методов** классификации.
2. Глубоко разобрать устройство, принцип обучения и применение **деревьев решений**.
3. Показать сильные стороны деревьев (интерпретируемость) и выявить их главную слабость (склонность к переобучению), подготавливая основу для изучения ансамблей.

### Часть 1: Введение в логические методы

В основе многих наших рассуждений лежит логика. Мы часто принимаем решения, последовательно отвечая на простые вопросы. Машинное обучение использует этот же принцип в **логических методах классификации**.

Представьте простую схему, которая помогает определить профиль человека:

```mermaid
graph TD;
    A{Использует ChatGPT?} -->|Да| B{Использует осознанно?};
    A -->|Нет| C{Делает вообще задания?};
    B -->|Да| D[<font color=green><b>Сдаст экзамен</b></font>];
    B -->|Нет| E[<font color=red><b>Не сдаст экзамен</b></font>];
    C -->|Да| F[<font color=green><b>Сдаст экзамен</b></font>];
    C -->|Нет| G[<font color=red><b>Не сдаст экзамен</b></font>];

    style D fill:#d4edda,stroke:#155724
    style F fill:#d4edda,stroke:#155724
    style E fill:#f8d7da,stroke:#721c24
    style G fill:#f8d7da,stroke:#721c24
```

Эта схема — это набор простых правил. Математически эти правила часто являются **конъюнкцией** (логическим "И") нескольких условий. Например, один из путей к провалу на экзамене можно записать так:

**ЕСЛИ** ("Использует ChatGPT?" = ДА) **И** ("Использует осознанно?" = НЕТ) **ТОГДА** (профиль = "Не сдаст экзамен")

В виде формулы это выглядит так:
$$ R(x) = [f_1(x)=1] \land [f_2(x)=0] \rightarrow \text{Не сдаст экзамен} $$

где $f_1(x)$ — признак "Использует ChatGPT?", а $f_2(x)$ — "Использует осознанно?".

#### От бинарных правил к вещественным признакам

В реальных данных признаки редко бывают бинарными (`да/нет`). Чаще они вещественные (температура, рост, цена). В этом случае логическое правило строится путем сравнения признака с некоторым **пороговым значением**.

Например, правило для классификации ирисов может выглядеть так: **ЕСЛИ** ("Длина лепестка" <= 2.45 см) **И** ("Ширина лепестка" > 1.8 см) **ТОГДА** (вид = "Virginica").

Математически это записывается с использованием нотации Айверсона:
$$ [f_j(x) \le a_j] $$
Это выражение равно `1` (истина), если значение $j$-го признака $f_j(x)$ для объекта $x$ меньше или равно порогу $a_j$, и `0` (ложь) в противном случае. Комбинируя такие простые правила, мы и строим **решающие деревья**.

#### Альтернативные логические подходы

Важно понимать, что решающие деревья — не единственный логический метод. Другой популярный подход, подсмотренный во врачебной практике, — это **синдромный подход**.

**Интуиция:** Врач ставит диагноз не по одному симптому, а по их совокупности. Если у пациента наблюдается достаточное количество (`d`) симптомов, характерных для определенной болезни, ставится соответствующий диагноз.

**Пример:** **ЕСЛИ** ((Температура > 38) + (Есть кашель) + (Болит горло) >= 2) **ТОГДА** (диагноз = "ОРВИ").

Математически это записывается так: мы подсчитываем, сколько признаков объекта $x$ выходят за рамки "нормы" (попадают в определенный интервал), и если это число превышает порог $d$, то правило срабатывает.

$$ R(x) = \left[ \sum_{j \in J} [a_j \le f_j(x) \le b_j] \ge d \right] $$

Хотя такие методы существуют, именно **решающие деревья** получили наибольшее распространение благодаря своей интерпретируемости и эффективности, поэтому далее мы сосредоточимся на них.

### Часть 2: Анатомия Дерева Решений

**Дерево решений** — это интуитивно понятная модель, представляющая собой ациклический граф, похожий на перевернутое дерево.

![tree](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/tree.png)

**Основные компоненты:**
*   **Корень (Root Node):** Самый верхний узел, с которого начинается разделение. В него попадают все объекты обучающей выборки.
*   **Внутренний узел (Internal Node):** Узел, который задает "вопрос" (проверяет условие) и разделяет данные на две или более ветви.
*   **Ветвь (Branch):** Линия, соединяющая узлы. Каждая ветвь соответствует одному из исходов проверки условия (например, "да" или "нет").
*   **Лист (Leaf / Terminal Node):** Конечный узел, в котором содержится итоговое решение (метка класса для классификации или среднее значение для регрессии).

**Процесс классификации:** Новый объект "проходит" по дереву от корня к листу, на каждом узле выбирая ветвь в зависимости от своих признаков. Класс, указанный в листе, в который он попал, и является предсказанием модели.

### Часть 3: Как дерево учится? Критерии качества разбиения

Главный вопрос при построении дерева: **Как в каждом узле найти лучший признак и лучший порог для разделения?**

Ответ: нужно выбрать такое правило, которое сделает дочерние узлы как можно более **"чистыми"** (однородными по составу классов).

#### Понятие "неопределенности" (Impurity)

"Неопределенность" — это мера того, насколько перемешаны классы в одном узле.

![impurity](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/impurity_examples.png)

Для численного измерения неопределенности используются специальные критерии. Два самых популярных — **критерий Джини** и **энтропия**.

**1. Критерий Джини (Gini Impurity):**
Показывает вероятность того, что случайно выбранный объект из узла будет неправильно классифицирован, если присвоить ему случайную метку из тех, что есть в этом узле.
$$ G = 1 - \sum_{k=1}^{K} p_k^2 $$
где $p_k$ — доля объектов класса $k$ в узле.
- $G=0$ для абсолютно "чистого" узла.
- $G$ стремится к максимуму для узла со смешанными классами.

**2. Энтропия (Entropy):**
Понятие из теории информации, которое измеряет хаос или неопределенность в системе.
$$ S = -\sum_{k=1}^{K} p_k \log_2 p_k $$
- $S=0$ для абсолютно "чистого" узла.
- $S$ максимальна, когда все классы представлены поровну.

![impurity](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/metrics.png)

#### Информационный выигрыш (Information Gain)
Дерево принимает решение о лучшем разбиении, максимизируя **информационный выигрыш**:
$$ IG = \text{Impurity}(\text{родитель}) - \sum_{j=1}^{m} \frac{N_j}{N} \text{Impurity}(\text{потомок}_j) $$
Алгоритм перебирает все возможные признаки и пороги, вычисляет для каждого `IG` и выбирает тот, который дает максимальное значение.

#### Пример: Пойдем ли мы играть в теннис?

Представим, что у нас есть 14 дней наблюдений, и мы хотим предсказать, пойдем ли мы играть в теннис. В корневом узле (до всяких разделений) у нас есть все 14 дней:
*   **9 дней**, когда мы пошли играть (`Play=Yes`)
*   **5 дней**, когда мы не пошли (`Play=No`)

**1. Расчет Критерия Джини (Gini Impurity) для корня:**
Показывает вероятность того, что мы ошибемся, если присвоим случайно выбранному дню случайную метку из имеющихся.
$$ G = 1 - \sum_{k=1}^{K} p_k^2 $$
Для нашего примера:
$$ G_{root} = 1 - ( (9/14)^2 + (5/14)^2 ) \approx 1 - (0.41 + 0.13) = 0.46 $$
Это наша исходная неопределенность.

**2. Расчет Энтропии (Entropy) для корня:**
Мера хаоса или неопределенности в системе.
$$ S = -\sum_{k=1}^{K} p_k \log_2 p_k $$
$$ S_{root} = - ( (9/14) \log_2(9/14) + (5/14) \log_2(5/14) ) \approx 0.94 $$

#### Как алгоритм находит лучший сплит? Информационный выигрыш (Жадный подход)

Алгоритм максимизирует **Информационный выигрыш (Information Gain)**. Это показатель того, насколько уменьшилась неопределенность после разбиения. **Алгоритм делает это для КАЖДОГО возможного сплита (по всем признакам и всем порогам) и выбирает тот, у которого `IG` максимален!**. Рассмотрим подробнее. 

Алгоритм построения дерева на каждом узле действует по **"жадному" (greedy)** принципу. Чтобы найти лучшее правило для разделения, он выполняет исчерпывающий, но эффективный поиск:

1.  **Цикл по всем признакам:** Алгоритм последовательно перебирает каждый доступный признак.
2.  **Поиск лучшего порога для признака:** Для текущего признака он проверяет все возможные пороговые значения (для числовых признаков) или все возможные комбинации разделения категорий (для категориальных). Для каждого такого потенциального сплита он вычисляет **Информационный выигрыш (Information Gain)**.
3.  **Выбор "чемпиона" признака:** Запоминается тот сплит (правило), который дал максимальный `IG` для данного признака.
4.  **Выбор абсолютного победителя:** После того как "чемпионы" найдены для **всех** признаков, алгоритм сравнивает их `IG` между собой и выбирает **единственное лучшее правило** — то, у которого `IG` оказался максимальным.

Именно это правило и используется для разделения текущего узла. Затем весь процесс рекурсивно повторяется для каждого из получившихся дочерних узлов.

**Пример: Пойдем ли мы играть в теннис?**

Представим, что мы находимся в **корневом узле** нашего будущего дерева. В нем все 14 дней наблюдений: **9 дней**, когда мы пошли играть (`Play=Yes`), и **5 дней**, когда не пошли (`Play=No`).

Неопределенность этого родительского узла по Gini:
$$ G_{root} = 1 - ( (9/14)^2 + (5/14)^2 ) \approx 1 - (0.413 + 0.128) = 0.459 $$

Теперь алгоритм должен выбрать, по какому признаку разделить эти 14 дней. Давайте сравним два варианта: "Ветер" и "Влажность".

**Вариант 1: Разделение по признаку "Ветер"**

```mermaid
graph TD;
    A[<b>Корень</b> <br> 14 дней <br> 9 Yes, 5 No <br> Gini=0.459] -->|Ветер=Слабый| B[<b>Потомок 1</b> <br> 8 дней <br> 6 Yes, 2 No <br> Gini=0.375];
    A -->|Ветер=Сильный| C[<b>Потомок 2</b> <br> 6 дней <br> 3 Yes, 3 No <br> Gini=0.5];
```

*   $G_{weak} = 1 - ((6/8)^2 + (2/8)^2) = 0.375$
*   $G_{strong} = 1 - ((3/6)^2 + (3/6)^2) = 0.5$

Средневзвешенная неопределенность потомков:
$$ \text{Weighted\_Gini}_{wind} = (\frac{8}{14}) \times 0.375 + (\frac{6}{14}) \times 0.5 \approx 0.214 + 0.214 = 0.428 $$

Информационный выигрыш для "Ветра":
$$ IG_{wind} = 0.459 - 0.428 = \mathbf{0.031} $$

**Вывод по Варианту 1:** Интуитивно это разбиение кажется не очень удачным. Один из потомков (`Wind=Strong`) такой же "грязный", как 50/50, а второй лишь ненамного "чище" родителя. Маленькое значение `IG=0.031` это и подтверждает.

---

**Вариант 2: Разделение по признаку "Влажность"**

Допустим, признак "Влажность" делит те же 14 дней на другие две группы:

```mermaid
graph TD;
    A[<b>Корень</b> <br> 14 дней <br> 9 Yes, 5 No <br> Gini=0.459] -->|Влажность=Высокая| B[<b>Потомок 1</b> <br> 7 дней <br> 3 Yes, 4 No <br> Gini=0.49];
    A -->|Влажность=Нормальная| C[<b>Потомок 2</b> <br> 7 дней <br> 6 Yes, 1 No <br> Gini=0.245];
```

*   $G_{high} = 1 - ((3/7)^2 + (4/7)^2) \approx 0.49$
*   $G_{normal} = 1 - ((6/7)^2 + (1/7)^2) \approx 0.245$

Средневзвешенная неопределенность потомков:
$$ \text{Weighted\_Gini}_{humidity} = (\frac{7}{14}) \times 0.49 + (\frac{7}{14}) \times 0.245 \approx 0.245 + 0.122 = 0.367 $$

Информационный выигрыш для "Влажности":
$$ IG_{humidity} = 0.459 - 0.367 = \mathbf{0.092} $$

---

**Принятие решения о сплите:**

Теперь алгоритм сравнивает информационный выигрыш от всех возможных сплитов. В нашем случае:

$$ IG_{humidity} (0.092) > IG_{wind} (0.031) $$

**Вывод:** Поскольку разделение по признаку "Влажность" дает большее уменьшение неопределенности, **алгоритм выберет именно "Влажность" в качестве первого сплита в дереве.** Этот процесс будет рекурсивно повторяться для каждого нового дочернего узла.

### Часть 3.1: Как именно перебираются сплиты?

#### Для числовых признаков:
Алгоритм не проверяет каждый возможный порог. Он делает это умнее:
1.  Берет все уникальные значения признака в текущем узле.
2.  Сортирует их по возрастанию.
3.  В качестве порогов для проверки берет **средние значения между соседними отсортированными точками**.
Например, если для признака "Температура" в узле есть значения `[10, 20, 30]`, алгоритм проверит пороги `15` и `25`.

#### Для категориальных признаков:
Если признак категориальный (например, "Город" со значениями `['Almaty', 'Astana', 'Karaganda']`), бинарное дерево должно разделить их на две группы. Алгоритм перебирает **все возможные комбинации** такого разделения:
*   `{Almaty}` vs `{Astana, Karaganda}`
*   `{Astana}` vs `{Almaty, Karaganda}`
*   `{Karaganda}` vs `{Almaty, Astana}`
...и для каждой комбинации считает Information Gain, чтобы найти лучшее бинарное разбиение.

### Часть 4: Главная проблема деревьев — Переобучение (Overfitting)

**Переобучение** — главная ахиллесова пята деревьев решений. Если не ограничивать рост дерева, оно будет продолжать делиться до тех пор, пока в каждом листе не останется по одному объекту. Такое дерево идеально запомнит обучающую выборку, включая весь шум, но будет очень плохо работать на новых, невиданных данных.

![overfitting](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/overfitting_tree_boundary.png)

**Как бороться с переобучением? (Регуляризация)**
Мы можем ограничить сложность дерева с помощью гиперпараметров. Этот процесс называется **pre-pruning** (предварительная обрезка).
- `max_depth`: максимальная глубина дерева.
- `min_samples_split`: минимальное количество объектов в узле для его дальнейшего разделения.
- `min_samples_leaf`: минимальное количество объектов, которое должно быть в листе.


### Часть 4.1: Альтернативный метод борьбы с переобучением — Усечение (Pruning)

Мы уже рассмотрели **pre-pruning** (или *early stopping*) — остановку роста дерева с помощью гиперпараметров (`max_depth` и т.д.). Однако есть и другой, часто более эффективный подход — **post-pruning** (последующее усечение или "стрижка").

**Идея:**
1.  Сначала мы строим **очень глубокое, заведомо переобученное дерево** без жестких ограничений на обучающей выборке.
2.  Затем мы начинаем "подстригать" его снизу вверх. Мы поочередно рассматриваем каждый узел (начиная с тех, что ближе к листьям) и задаем вопрос: "А что, если мы **удалим** это разделение (этот узел) и превратим его в лист?".
3.  Мы **сравниваем качество** модели до и после "стрижки" на **отложенной (валидационной) выборке**.
4.  Если удаление узла (и всего поддерева под ним) **не ухудшает или даже улучшает** качество на валидационной выборке, то мы **выполняем "стрижку"**.

**Простыми словами:** Мы сначала строим слишком сложную модель, а затем "отрезаем" от нее те части, которые, скорее всего, просто выучили шум в обучающих данных и не несут реальной пользы для обобщения.

<img src="https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/ShouldWePrune.png" width="500" height="400">
<img src="https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/pruneddecisiontree.png" width="800" height="900">

**Почему Post-Pruning может быть лучше?**
Pre-pruning — это "жадный" подход. Он может остановить рост дерева слишком рано. Например, какое-то разделение может сейчас казаться бесполезным, но оно откроет путь к очень хорошим разделениям на следующих уровнях. Pre-pruning никогда не даст дереву дойти до этих хороших сплитов.

Post-pruning же видит всю картину целиком и принимает более информированное решение о том, какие ветви действительно являются избыточными.

В `scikit-learn` основной метод регуляризации — это **pre-pruning** через гиперпараметры. Однако, в `DecisionTreeClassifier` есть параметр `ccp_alpha` (Cost-Complexity Pruning), который реализует один из видов **post-pruning**. Это более продвинутая техника, которая позволяет найти оптимальный баланс между сложностью дерева и его точностью.

### Часть 5: Деревья для Регрессии

Деревья можно использовать и для задач регрессии.

**Принцип работы:**
1.  **Критерий разбиения:** Вместо уменьшения неопределенности (Gini/Entropy), дерево ищет такой сплит, который максимально уменьшает **среднеквадратичную ошибку (MSE)**. Для родительского узла $R_m$ и двух его потомков $R_l$ (левый) и $R_r$ (правый), алгоритм максимизирует следующий информационный выигрыш:

    $$ Q(R_m, j, t) = H(R_m) - \frac{|R_l|}{|R_m|}H(R_l) - \frac{|R_r|}{|R_m|}H(R_r) \rightarrow \max $$

    Где в качестве меры неопределенности $H(R)$ выступает дисперсия или MSE относительно среднего значения в узле. По сути, дерево ищет сплит, который делает значения в дочерних узлах как можно более "скученными".

2.  **Предсказание в листе:** В каждом листе $v$ хранится не метка класса, а одно константное значение $b_v$. Это значение является **средним арифметическим** всех целевых переменных $y_i$ объектов, попавших в этот лист:

    $$ b_v = \frac{1}{|R_v|} \sum_{i \in R_v} y_i $$

    Именно это и приводит к тому, что предсказание дерева регрессии выглядит как **ступенчатая функция**. Каждая "ступенька" — это среднее значение в одном из листьев.


В результате предсказание дерева регрессии выглядит как **ступенчатая функция**.

**Пример:** Аппроксимация функции `cos(x)`.
Представим, что у нас есть точки, лежащие на кривой косинуса. Дерево регрессии будет пытаться приблизить эту гладкую кривую с помощью горизонтальных отрезков.

<table>
  <tr>
    <td><img src="https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/cos1d.png" alt="1" width="400"></td>
    <td><img src="https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/cos2d.png" alt="2" width="400"></td>
    <td><img src="https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/cos3.png" alt="3" width="400"></td>
  </tr>
</table>

С увеличением глубины дерева "ступеньки" становятся все меньше, и модель начинает подгоняться не только под саму функцию, но и под случайный шум в данных, что также приводит к переобучению.

### Часть 6: Практическая реализация в Python

#### 6.1. Пример Классификации (Датасет Iris)

Давайте построим простое дерево для классической задачи определения вида ириса по его параметрам.

In [ ]:
# Решающее дерево для задачи классификации

import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import datasets


def get_grid(data):
    x_min, x_max = data[:, 0].min() - 1, data[:, 0].max() + 1
    y_min, y_max = data[:, 1].min() - 1, data[:, 1].max() + 1
    return np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))


iris = datasets.load_iris()
train_data = np.c_[iris.data[:, 0].reshape(-1, 1), iris.data[:, 2].reshape(-1, 1)]
train_labels = iris.target

clf_tree = DecisionTreeClassifier(criterion='entropy', max_depth=2, random_state=5)
clf_tree.fit(train_data, train_labels)




xx, yy = get_grid(train_data)
predicted = clf_tree.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
plt.pcolormesh(xx, yy, predicted, cmap='spring', shading='auto')
plt.scatter(train_data[:, 0], train_data[:, 1], c=train_labels, s=50, cmap='spring', edgecolors='black', linewidth=1.5)
plt.show()
plt.figure(figsize=(14,14))
plot_tree(clf_tree,feature_names=iris.feature_names,  
          class_names=iris.target_names,
          filled=True)
plt.show()

#### 6.2. Пример Регрессии (Аппроксимация косинуса)

Теперь реализуем пример, который мы обсуждали в теории: аппроксимация функции `cos(x)`.

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

# Генерируем данные
rng = np.random.RandomState(1)
X = np.sort(5 * rng.rand(80, 1), axis=0)
y = np.sin(X).ravel()
y[::5] += 3 * (0.5 - rng.rand(16)) # Добавляем шум

# Обучаем дерево регрессии
regr = DecisionTreeRegressor(max_depth=2)
regr.fit(X, y)

# Делаем предсказания
X_test = np.arange(0.0, 5.0, 0.01)[:, np.newaxis]
y_pred = regr.predict(X_test)

# Визуализируем результат
plt.figure(figsize=(10, 7))
plt.scatter(X, y, s=20, edgecolor="black", c="darkorange", label="Данные")
plt.plot(X_test, y_pred, color="cornflowerblue", label="Предсказание (max_depth=3)", linewidth=2)
plt.xlabel("x")
plt.ylabel("y")
plt.title("Аппроксимация функции деревом регрессии")
plt.legend()
plt.grid()
plt.show()


# Визуализируем дерево
plt.figure(figsize=(12, 8))
plot_tree(regr, 
         filled=True)
plt.title("Дерево решений для задачи регрессии)")
plt.show()

### Часть 7: Преимущества и недостатки Деревьев Решений

#### Преимущества:
*   **Высокая интерпретируемость:** Логику дерева легко понять и объяснить даже неспециалисту. Это "белый ящик".
*   **Не требуют масштабирования данных:** Работают с признаками в их исходном масштабе.
*   Могут работать как с числовыми, так и с категориальными признаками.

#### Недостатки:
*   **Склонность к переобучению:** Требуют тщательной настройки гиперпараметров (регуляризации).
*   **Нестабильность:** Небольшие изменения в обучающих данных могут привести к построению совершенно другого дерева.